In [62]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

# Make team names uniform between sheets

In [ ]:
names_file ='/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/names.csv'

In [ ]:
df = pd.read_csv(names_file)

In [ ]:
mapping_dict = {key: value for key, value in zip(df['abbrev'], df['full_name'])}

In [ ]:
extra_data_file = '/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_extra_stats.csv'

In [ ]:
extra_df = pd.read_csv(extra_data_file)

In [ ]:
extra_df = extra_df.replace(mapping_dict)

In [ ]:
extra_data_file_new = '/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_extra_stats_uniform_names.csv'

In [ ]:
extra_df.to_csv(extra_data_file_new)

# Step 2: Convert all times to MST

In [63]:
raw = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_full.csv')

/Users/stefanfeiler/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (90,92,94,96,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [64]:
# # This takes a few minutes
time_columns = [col for col in raw.columns if 'time' in col]
raw[time_columns] = raw[time_columns].astype('datetime64[ns]')

In [65]:
raw[time_columns] = raw[time_columns] - pd.Timedelta(hours=7)

# Step 3: Make the same my_game_id values in each sheet

In [66]:
extra = pd.read_csv(extra_data_file_new)

In [68]:
raw['date'] = raw['commence_time'].dt.strftime('%Y%m%d')

In [69]:
raw['date']

0         20221105
1         20221105
2         20221105
3         20221105
4         20221105
            ...   
921709    20200722
921710    20200722
921711    20200722
921712    20200722
921713    20200722
Name: date, Length: 921714, dtype: object

In [70]:
raw['my_id'] = raw['home_team'] + raw['away_team'] + raw['date']
raw['my_id']

0         Houston AstrosPhiladelphia Phillies20221105
1         Houston AstrosPhiladelphia Phillies20221105
2         Houston AstrosPhiladelphia Phillies20221105
3         Houston AstrosPhiladelphia Phillies20221105
4         Houston AstrosPhiladelphia Phillies20221105
                             ...                     
921709     Chicago White SoxMilwaukee Brewers20200722
921710     Los Angeles AngelsSan Diego Padres20200722
921711            Chicago CubsMinnesota Twins20200722
921712        Boston Red SoxToronto Blue Jays20200722
921713          Colorado RockiesTexas Rangers20200722
Name: my_id, Length: 921714, dtype: object

In [74]:
extra['date']= extra['date'].astype(str)
extra['my_id'] = extra['home_team'] + extra['away_team'] + extra['date']
extra['my_id']

0       Los Angeles DodgersSan Francisco Giants20200723
1          Washington NationalsNew York Yankees20200723
2                 Chicago CubsMilwaukee Brewers20200724
3                 Cincinnati RedsDetroit Tigers20200724
4       Los Angeles DodgersSan Francisco Giants20200724
                             ...                       
5882        Houston AstrosPhiladelphia Phillies20221029
5883        Philadelphia PhilliesHouston Astros20221101
5884        Philadelphia PhilliesHouston Astros20221102
5885        Philadelphia PhilliesHouston Astros20221103
5886        Houston AstrosPhiladelphia Phillies20221105
Name: my_id, Length: 5887, dtype: object

# Step 4: Make a complicated dict

In [118]:
# Make a dict that maps my_id to game_id and includes duplicates
mapping_dict = {}
for key, value in zip(raw['my_id'], raw['game_id']):
    if key in mapping_dict:
        mapping_dict[key].append(value)  # Append value to existing key's list
        mapping_dict[key] = list(set(mapping_dict[key]))
    else:
        mapping_dict[key] = [value]  # Create a new key-value pair

In [119]:
# Make a dict that maps game_id's to commence_times
id_commence_dict = {key: value for key, value in zip(raw['game_id'], raw['commence_time'])}

In [121]:
# Delete all double and triple headers
new_dict = {key: value for key, value in mapping_dict.items() if len(value) == 1}

8
